In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import string
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'
from dotenv import load_dotenv
import os

load_dotenv()  

True

In [2]:
hf_token = os.getenv("HF_TOKEN")

# Set up the authorization token for Hugging Face
os.environ["HF_HOME"] = os.path.expanduser("~/.cache/huggingface")
os.environ["TRANSFORMERS_CACHE"] = os.path.expanduser("~/.cache/huggingface/transformers")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.expanduser("~/.cache/huggingface/hub")
os.environ["HUGGINGFACE_CO_TOKEN"] = hf_token  # Ensure this is set for authorization

In [3]:
df_ip_5_mind = pd.read_csv('../data/In-person 5-day Residential_Mindfulness_for_Health_2024-02-22_11_16_04.csv')

In [4]:
df_ip_5_mind.rename(columns={
    'Submission Date': 'submission_date',
    'Last Update Date': 'last_update_date',
    'Course Title/Date': 'course_title_date',
    'City': 'city',
    'State / Province': 'state_provence',
    'Country': 'country',
    'Date of Birth': 'date_of_birth',
    'Gender': 'gender',
    'Ethnicity': 'ethnicity'
},
                    inplace=True)

In [5]:
df_ip_5_mind_1 = pd.read_csv(
    '../data/In-person Mindfulness_for_Health_2024-02-22_11_22_19.csv')

In [6]:
df_ip_5_mind_2 = pd.read_csv('../data/In-person Mindfulness_for_Stress_Booking_2024-02-22_11_16_48.csv')

In [7]:
df_ip_5_mind_3 = pd.read_csv('../data/Online_Mindfulness_for_Health_C2024-02-22_11_12_22.csv')

In [8]:
df_ip_5_mind_4 = pd.read_csv('../data/Online_Mindfulness_for_Stress_B2024-02-22_11_15_21.csv')

In [9]:
df_ip_5_mind.columns

Index(['submission_date', 'last_update_date', 'course_title_date', 'city',
       'state_provence', 'country', 'date_of_birth', 'gender', 'ethnicity',
       'What has brought you to choose this course? Please outline briefly.',
       'What condition(s) do you have that has brought you to this course?',
       'Do you experience acute or debilitating depression or any other mental health condition(s)? If so, can you please say a little more about this?',
       'Are you OK to work alongside others in a group situation? (Please note that group size varies)',
       'Are you able to practice mindfulness exercises/techniques for at least 20 minutes at a time for the duration of the course?',
       'Do you have any particular needs that it would be useful for us to know about, e.g. restricted mobility, hearing or visual impairment?',
       'Please add any other comments here',
       'If you are happy to, please state whether you currently identify as having a chronic pain condition (i.

In [10]:
df = pd.read_csv('../data/omfh_backup.csv')

In [11]:
df.Motivation

0                                                                                                                                                                                                                                                                                                                                          I'm an occupational therapist working in the NHS with patients with Long Covid. I use basic Mindfulness relaxation in my sessions but would like to work towards gaining deeper knowledge and accreditation. 
1                                                                                                                                                                                                                                                                                                                          I'm joining this course in the hopes that I cn find some useful techniques to help me deal with chronic migraines, neck and knee pain. I would like 

In [12]:
df

,CourseDate,CourseType,EnrollmentDate,Gender,Ethnicity,DoB,PersonalHistory,Motivation,Are you able to practice mindfulness exercises/techniques for at least 20 minutes per day for the duration of the course?,No Label,Is your organisation paying for the course?,No Label.1,City,County,Which of these have you done with Breathworks so far? (tick any that apply),"If you have particular access or communication requirements which need to be met in order for you to engage fully with this online retreat, please let us know.",Is someone else paying for your course (not including employer)?,"If Yes, please provide the name of a person that appears on a receipt",How did you hear about Breathworks?
0,2024-04-03,OMfH,2024-02-22,Female,"White (including White British, Irish, Gypsy or Irish Traveller)",08/03/1967,"I have issues with fatigue and stress. No, currently I don't identify as having a chronic pain condition. No, I don't experience acute or debilitating depression or any other mental health condition(s).",I'm an occupational therapist working in the NHS with patients with Long Covid. I use basic Mindfulness relaxation in my sessions but would like to work towards gaining deeper knowledge and accreditation.,Yes,Ongoing support communications sign up,Yes,I would like to receive updates from Breathworks,Magdalen Square,Liverpool,Signed up to the newsletter\nJoined the Breathworks Online Community of Practice,No requirements.,No,NaN,Website
1,2024-06-02,OMfH,2024-02-22,Female,"White (including White British, Irish, Gypsy or Irish Traveller)",29/11/1981,"Migraines, disc hernia in neck, undiagnosed knee pain Yes, I have experienced persistent pain that has lasted for at least the last 3 months. I spent 4 years fighting a near ftal depression. Two years of which were total blackness. I am now fully recovered and aiming to be fully off of anti-depressants within the next couple of months.","I'm joining this course in the hopes that I cn find some useful techniques to help me deal with chronic migraines, neck and knee pain. I would like to be able to accept the pain better and learn to fight less against it.",Yes,Ongoing support communications sign up,No,I would like to receive updates from Breathworks,NaN,NaN,Signed up to the newsletter,NaN,No,NaN,Internet search
2,2024-03-07,OMfH,2024-02-20,Female,"White (including White British, Irish, Gypsy or Irish Traveller)",07/03/1975,"Auto-immune issues (experience frequent gastric, sinus, head issues as though they are allergic or post viral reactions) for several years, more recently added vestibular issues arising during perimenopause and now continuing post menopause. No, currently I don't identify as having a chronic pain condition. Yes anxiety which is new for me, occasional panic attack.","Always been high stress and consistently advised to do something about it, I'm a full-time working, single parent to 2 young adults with ASD/Anxiety/ADD, there is no time or resource to de-stress from this.\nI want to live, find ways to reduce stress and my active thoughts and listen to my body rather than it rule what speed I can go at or how exhausted I get when I push through. Easy right?! \nI'm finally making 2024 the year of healing for myself and I think mindfulness is a great skill to add to help me reach my goal.\n",Yes,Ongoing support communications sign up,No,NaN,NaN,NaN,NaN,NaN,No,NaN,Internet search
3,2024-03-07,OMfH,2024-02-20,Female,"White (including White British, Irish, Gypsy or Irish Traveller)",09/06/1974,"FMS and long covid Yes, I have experienced persistent pain that has lasted for at least the last 3 months. no, I don't experience acute or debilitating depression or any other mental health condition(s).","I am looking forward to joining this course as i have both FMS and also long Covid.\n\nI have been very influenced by therapists i have seen in helping me to look at my life differently and include more relaxation and mindfulness into my day to day life, this has help control my pain, d

In [13]:
df.Motivation.fillna('',inplace=True)

In [14]:
def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    cleaned = ' '.join(lemmatized) # Join back to a string
    return cleaned

# Apply to all texts
df['clean_text'] = df.Motivation.apply(clean)

In [15]:
df

,CourseDate,CourseType,EnrollmentDate,Gender,Ethnicity,DoB,PersonalHistory,Motivation,Are you able to practice mindfulness exercises/techniques for at least 20 minutes per day for the duration of the course?,No Label,Is your organisation paying for the course?,No Label.1,City,County,Which of these have you done with Breathworks so far? (tick any that apply),"If you have particular access or communication requirements which need to be met in order for you to engage fully with this online retreat, please let us know.",Is someone else paying for your course (not including employer)?,"If Yes, please provide the name of a person that appears on a receipt",How did you hear about Breathworks?,clean_text
0,2024-04-03,OMfH,2024-02-22,Female,"White (including White British, Irish, Gypsy or Irish Traveller)",08/03/1967,"I have issues with fatigue and stress. No, currently I don't identify as having a chronic pain condition. No, I don't experience acute or debilitating depression or any other mental health condition(s).",I'm an occupational therapist working in the NHS with patients with Long Covid. I use basic Mindfulness relaxation in my sessions but would like to work towards gaining deeper knowledge and accreditation.,Yes,Ongoing support communications sign up,Yes,I would like to receive updates from Breathworks,Magdalen Square,Liverpool,Signed up to the newsletter\nJoined the Breathworks Online Community of Practice,No requirements.,No,NaN,Website,occupational therapist working nh patient long covid use basic mindfulness relaxation session would like work towards gaining deeper knowledge accreditation
1,2024-06-02,OMfH,2024-02-22,Female,"White (including White British, Irish, Gypsy or Irish Traveller)",29/11/1981,"Migraines, disc hernia in neck, undiagnosed knee pain Yes, I have experienced persistent pain that has lasted for at least the last 3 months. I spent 4 years fighting a near ftal depression. Two years of which were total blackness. I am now fully recovered and aiming to be fully off of anti-depressants within the next couple of months.","I'm joining this course in the hopes that I cn find some useful techniques to help me deal with chronic migraines, neck and knee pain. I would like to be able to accept the pain better and learn to fight less against it.",Yes,Ongoing support communications sign up,No,I would like to receive updates from Breathworks,NaN,NaN,Signed up to the newsletter,NaN,No,NaN,Internet search,joining course hope cn find useful technique help deal chronic migraine neck knee pain would like able accept pain better learn fight le
2,2024-03-07,OMfH,2024-02-20,Female,"White (including White British, Irish, Gypsy or Irish Traveller)",07/03/1975,"Auto-immune issues (experience frequent gastric, sinus, head issues as though they are allergic or post viral reactions) for several years, more recently added vestibular issues arising during perimenopause and now continuing post menopause. No, currently I don't identify as having a chronic pain condition. Yes anxiety which is new for me, occasional panic attack.","Always been high stress and consistently advised to do something about it, I'm a full-time working, single parent to 2 young adults with ASD/Anxiety/ADD, there is no time or resource to de-stress from this.\nI want to live, find ways to reduce stress and my active thoughts and listen to my body rather than it rule what speed I can go at or how exhausted I get when I push through. Easy right?! \nI'm finally making 2024 the year of healing for myself and I think mindfulness is a great skill to add to help me reach my goal.\n",Yes,Ongoing support communications sign up,No,NaN,NaN,NaN,NaN,NaN,No,NaN,Internet search,always high stress consistently advised something full time working single parent young adult asd anxiety add time resource de stress want live find way reduce stress active thought listen body rather rule speed go exhausted get push easy right finally making year healing think mindfulness great skill 

In [16]:
vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(df['clean_text'])

lda_model = LatentDirichletAllocation(n_components=3)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [17]:
lda_vectors.shape

(1399, 3)

In [18]:
def get_topics_as_dict(model, vectorizer):
    topics = {}
    for idx, topic in enumerate(model.components_):
        topics[f"topic_{idx}"] = [(vectorizer.get_feature_names_out()[i], topic[i])
                                  for i in topic.argsort()[:-10 - 1:-1]]
    return topics

# Usage
# lda_model = your LDA model
# vectorizer = your vectorizer instance
# topics_dict = get_topics_as_dict(lda_model, vectorizer)


In [19]:
topics = get_topics_as_dict(lda_model, vectorizer)

In [20]:
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

# Assuming `lda_model` is your trained LatentDirichletAllocation model
# and `vectorizer` is the CountVectorizer or TfIdfVectorizer used to transform your documents

panel = pyLDAvis.lda_model.prepare(lda_model,data_vectorized, vectorizer, mds='tsne')
pyLDAvis.display(panel)

In [21]:
data_vectorized.shape

(1399, 3367)

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


/Users/nils/.pyenv/versions/3.10.6/envs/breathworks/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

/Users/nils/.pyenv/versions/3.10.6/envs/breathworks/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning:

Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.



In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming `features` is your combined matrix of demographic data and LDA topic distributions
scaler = StandardScaler()
features_scaled = scaler.fit_transform(lda_vectors)

kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(features_scaled)

# Add the cluster assignments back to your original DataFrame
df['Cluster'] = clusters

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline
pyo.init_notebook_mode()# Assuming 'features_scaled' is your scaled 3D feature array and 'df' contains a 'Cluster' column
fig = px.scatter_3d(df, x=features_scaled[:, 0], y=features_scaled[:, 1], z=features_scaled[:, 2],
                    color='Cluster', title='3D Cluster Visualization')
fig.show()


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b",)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
response = generator('whats a your favourite thing to do on a first date?',num_return_sequences=5)

In [ ]:
response[0]